In [1]:
#######
# 영화 후기 데이터 -> NNLM 모델 적용 
#######

### 데이터 읽기
import pandas as pd
from pandas import DataFrame, Series
df_train = pd.read_csv('100 data/ratings_test.txt', delimiter='\t', keep_default_na=False)
print(df_train['document'][:10])

# Data는 100개만 사용
text = df_train['document'][:100]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object


In [2]:
### 형태소 분석 및 one-hot encoding
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
twitter_tag = Okt()

# 형태소 분석 방법(조사, 어미 등 제외)
def twitter_tokenizer_part(text):
    lst = []
    for tpl in twitter_tag.pos(text, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            lst.append(tpl[0])
    if len(lst) != 0: 
        return lst
    else:
        return [""]
    
# CountVectorizer 객체 형성 -> one-hot encoding
vect = CountVectorizer(tokenizer=twitter_tokenizer_part).fit(text)
print(vect.get_feature_names()[80:90])
print(len(vect.get_feature_names()))

['괜히', '굉장하다', '교훈', '구성', '군', '굳다', '굿굿', '궁금', '귀엽다', '그']
622


In [3]:
# 입력할 후기를 토큰화함
tokenized_words = []
for line in text:
    for tpl in twitter_tag.pos(line, stem=True):
        if not tpl[1] in ["Josa", "Eomi", "PreEomi", "Punctuation"]:
            tokenized_words.append(tpl[0])

print(tokenized_words[10:20])
print(len(tokenized_words))

['점', '짜다', '리', '더', '더욱', '아니다', '지루하다', '않다', '완전', '막장']
981


In [4]:
# n-1, n-2, n-3번째 단어를 통해 n번째 단어를 예측하는 학습 데이터 생성
x_train = []
y_train = []
for i in range(len(tokenized_words)-3):
    x_train.append(vect.transform([tokenized_words[i],tokenized_words[i+1],tokenized_words[i+2]]).toarray())
    y_train.append(vect.transform([tokenized_words[i+3]]).toarray())
    
# 데이터 예시
y_train[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [5]:
# 리스트를 array로 변형
import numpy as np
x_train = np.array(x_train).reshape(978, 3, 622)
y_train = np.array(y_train).reshape(978, 622)

In [6]:
### 모델 생성 
from keras.models import Model
from keras.layers import Dense, Input, Flatten
from keras.optimizers import SGD

Using TensorFlow backend.


In [7]:
# 방식1: one-hot encoding 형태로 입력
x = Input([3,len(y_train[0])])
x2 = Dense(2)(x)
# 방식2: index 형태로 입력 
# x = Input((3,))
# x2 = Embedding(y_train[0], 2)(x)
x3 = Flatten()(x2)
h = Dense(64, activation='tanh')(x3)
y = Dense(len(y_train[0]), activation='softmax')(h)
model = Model(x,y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 622)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 3, 2)              1246      
_________________________________________________________________
flatten_1 (Flatten)          (None, 6)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                448       
_________________________________________________________________
dense_3 (Dense)              (None, 622)               40430     
Total params: 42,124
Trainable params: 42,124
Non-trainable params: 0
_________________________________________________________________


In [8]:
sgd = SGD(lr=0.1) # default = 0.01
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=1000, batch_size=100)

Epoch 1/1000
978/978 [==============================] - 1s 648us/step - loss: 6.3271 - acc: 0.0082
Epoch 2/1000
978/978 [==============================] - 0s 280us/step - loss: 6.3240 - acc: 0.0123
Epoch 3/1000
978/978 [==============================] - 0s 287us/step - loss: 6.3209 - acc: 0.0204
Epoch 4/1000
978/978 [==============================] - 0s 276us/step - loss: 6.3178 - acc: 0.0204
Epoch 5/1000
978/978 [==============================] - 0s 266us/step - loss: 6.3147 - acc: 0.0235
Epoch 6/1000
978/978 [==============================] - 0s 288us/step - loss: 6.3116 - acc: 0.0174
Epoch 7/1000
978/978 [==============================] - 0s 260us/step - loss: 6.3085 - acc: 0.0143
Epoch 8/1000
978/978 [==============================] - 0s 275us/step - loss: 6.3052 - acc: 0.0215
Epoch 9/1000
978/978 [==============================] - 0s 261us/step - loss: 6.3020 - acc: 0.0153
Epoch 10/1000
978/978 [==============================] - 0s 273us/step - loss: 6.2987 - acc: 0.0174
Epoch 11/

978/978 [==============================] - 0s 364us/step - loss: 1.6998 - acc: 0.7423
Epoch 889/1000
978/978 [==============================] - 0s 308us/step - loss: 1.6960 - acc: 0.7434
Epoch 890/1000
978/978 [==============================] - 0s 281us/step - loss: 1.6924 - acc: 0.7444
Epoch 891/1000
978/978 [==============================] - 0s 350us/step - loss: 1.6871 - acc: 0.7423 0s - loss: 1.7252 - acc: 0.73
Epoch 892/1000
978/978 [==============================] - 0s 273us/step - loss: 1.6818 - acc: 0.7434
Epoch 893/1000
978/978 [==============================] - 0s 264us/step - loss: 1.6793 - acc: 0.7485
Epoch 894/1000
978/978 [==============================] - 0s 277us/step - loss: 1.6734 - acc: 0.7474
Epoch 895/1000
978/978 [==============================] - 0s 285us/step - loss: 1.6719 - acc: 0.7495
Epoch 896/1000
978/978 [==============================] - 0s 259us/step - loss: 1.6675 - acc: 0.7505
Epoch 897/1000
978/978 [==============================] - 0s 279us/step - lo

In [11]:
pre = model.predict(x_train)

In [38]:
for i in range(43,99):
    print(tokenized_words[i: i+3], end='===>')
    print(pre[i].argmax(), end=' ')
    print(vect.get_feature_names()[pre[i].argmax()])

['되다', '최고', '음악']===>416 영화
['최고', '음악', '영화']===>533 진정하다
['음악', '영화', '진정하다']===>352 쓰레기
['영화', '진정하다', '쓰레기']===>216 마치
['진정하다', '쓰레기', '마치']===>255 미국
['쓰레기', '마치', '미국']===>379 애니
['마치', '미국', '애니']===>578 튀어나오다
['미국', '애니', '튀어나오다']===>595 하다
['애니', '튀어나오다', '한']===>549 창의력
['튀어나오다', '한', '창의력']===>398 없다
['한', '창의력', '없다']===>202 로봇
['창의력', '없다', '로봇']===>188 디자인
['없다', '로봇', '디자인']===>72 고개
['로봇', '디자인', '고개']===>500 젖다
['디자인', '고개', '젖다']===>595 하다
['고개', '젖다', '하다']===>47 갈수록
['젖다', '하다', '갈수록']===>57 개판
['하다', '갈수록', '개판']===>178 되다
['갈수록', '개판', '되다']===>521 중국영화
['개판', '되다', '중국영화']===>440 유치하다
['되다', '중국영화', '유치하다']===>416 영화
['중국영화', '유치하다', '내용']===>398 없다
['유치하다', '내용', '없다']===>588 폼
['내용', '없다', '폼']===>483 잡다
['없다', '폼', '잡다']===>115 끝나다
['폼', '잡다', '끝나다']===>185 들다
['잡다', '끝나다', '말']===>595 하다
['끝나다', '말', '안되다']===>244 무기
['말', '안되다', '무기']===>440 유치하다
['안되다', '무기', '유치하다']===>19 cg
['무기', '유치하다', 'cg']===>128 남무
['유치하다', 'cg', '남무']===>512 좋다
['cg', '남무', '아']==

In [13]:
# 워드임배딩 잘됐는지 확인 
embedded_W = model.get_weights()[0]
print(embedded_W.shape)

(622, 2)


In [14]:
# 코사인 유사도 함수 의 
def cosine_similarity(A, B):
    if np.sum(A*B) == 0:
        return 0
    return np.sum(A*B) / (np.sqrt(np.sum(A*A)) * np.sqrt(np.sum(B*B)))

In [30]:
print(vect.get_feature_names()[416])
print(vect.get_feature_names()[352])
print(vect.get_feature_names()[587])

영화
쓰레기
포켓몬스터


In [29]:
# 영화와 쓰레기의 코사인 유사도 
A = embedded_W[416]
B = embedded_W[352]
cosine_similarity(A,B)

0.47847405

In [31]:
# 영화와 포켓몬스터의 코사인 유사도 
A = embedded_W[416]
B = embedded_W[587]
cosine_similarity(A,B)

-0.068978064